In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import sys

sys.path.append("..")

# Import utilites
from utils import label_map_util
from utils import visualization_utils as vis_util

C:\Users\dso1222\anaconda3\envs\tensorflow_cpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dso1222\anaconda3\envs\tensorflow_cpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dso1222\anaconda3\envs\tensorflow_cpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dso1222\anaconda3\envs\tensorfl

In [2]:
MODEL_NAME = 'inference_graph'
VIDEO_NAME = 'test.mov'

In [3]:
PATH_TO_CKPT = os.path.abspath(r'C:\Users\dso1222\Documents\watchtower\Tensorflow\workspace_dumpster\training_demo\inference_graph\frozen_inference_graph.pb')

PATH_TO_LABELS = os.path.abspath(r'C:\Users\dso1222\Documents\watchtower\Tensorflow\workspace_dumpster\training_demo\annotations\label_map.pbtxt')

PATH_TO_VIDEO_FOLDER = os.path.abspath(r'C:\Users\dso1222\Documents\watchtower\Tensorflow\workspace_dumpster\training_demo\TestVideo')

NUM_CLASSES = 2

In [4]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

In [6]:
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
num_detections = detection_graph.get_tensor_by_name('num_detections:0')



In [7]:
for video_name in os.listdir(PATH_TO_VIDEO_FOLDER):
    print(video_name)

CardboardFront.mov
CardboardFront1.mov
CartFront1.mov
CartFront2.mov
CartFront3.mov
CartSide.mov
CartSide2.mov
CartSide3.mov
test.mov


In [9]:
video = cv2.VideoCapture(f'{PATH_TO_VIDEO_FOLDER}/CartFront1.mov')
bounding_box =[]
while(video.isOpened()):
    ret, frame = video.read()
    frame_expanded = np.expand_dims(frame, axis=0)

    (boxes, scores, classes, num) = sess.run(
        [detection_boxes, detection_scores, detection_classes, num_detections],
        feed_dict={image_tensor: frame_expanded})

    vis_util.visualize_boxes_and_labels_on_image_array(
        frame,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8,
        min_score_thresh=0.60)
    
    coordinates = vis_util.return_coordinates(
                frame,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8,
                min_score_thresh=0.80)
    print(coordinates)
    
    cv2.namedWindow('Object detector', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Object detector', 1300, 1000)
    cv2.imshow('Object detector', frame)
#     print(boxes)
    
    
    # Press 'q' to quit
    if cv2.waitKey(1) == ord('q'):
        break
video.release()
cv2.destroyAllWindows()

[[431, 818, 610, 1104, 99.73264336585999, ['Dumpster: 99%']], [461, 930, 1380, 1611, 92.84646511077881, ['Person: 92%']]]
[[428, 819, 608, 1107, 99.59614276885986, ['Dumpster: 99%']], [448, 908, 1376, 1620, 95.00068426132202, ['Person: 95%']]]
[[426, 819, 611, 1105, 99.36834573745728, ['Dumpster: 99%']], [454, 934, 1379, 1607, 96.83652520179749, ['Person: 96%']]]
[[418, 819, 619, 1099, 98.99320602416992, ['Dumpster: 98%']], [447, 917, 1373, 1590, 95.01233100891113, ['Person: 95%']]]
[[421, 821, 618, 1102, 98.95820021629333, ['Dumpster: 98%']], [440, 922, 1374, 1588, 98.31085205078125, ['Person: 98%']]]
[[417, 814, 616, 1099, 99.41073656082153, ['Dumpster: 99%']], [444, 926, 1369, 1588, 98.35406541824341, ['Person: 98%']]]
[[419, 814, 619, 1100, 99.40218925476074, ['Dumpster: 99%']], [444, 919, 1364, 1584, 99.10041093826294, ['Person: 99%']]]
[[430, 819, 616, 1103, 99.62965250015259, ['Dumpster: 99%']], [454, 916, 1361, 1580, 99.12697076797485, ['Person: 99%']]]
[[434, 821, 615, 1098, 9

[[418, 786, 674, 1086, 99.61707592010498, ['Dumpster: 99%']], [438, 882, 1215, 1425, 95.5280601978302, ['Person: 95%']]]
[[417, 786, 674, 1088, 99.34785962104797, ['Dumpster: 99%']], [451, 883, 1226, 1429, 95.69782018661499, ['Person: 95%']]]
[[416, 791, 677, 1084, 99.06986355781555, ['Dumpster: 99%']], [435, 865, 1215, 1437, 88.31422328948975, ['Person: 88%']]]
[[410, 790, 680, 1095, 99.26007390022278, ['Dumpster: 99%']]]
[[402, 793, 683, 1085, 98.71096014976501, ['Dumpster: 98%']]]
[[392, 792, 684, 1077, 96.27094268798828, ['Dumpster: 96%']]]
[[402, 791, 688, 1078, 93.53447556495667, ['Dumpster: 93%']]]
[[401, 790, 692, 1095, 97.35894203186035, ['Dumpster: 97%']], [437, 864, 1223, 1406, 91.07288122177124, ['Person: 91%']]]
[[386, 784, 684, 1082, 89.32338953018188, ['Dumpster: 89%']], [416, 864, 1224, 1399, 84.96578335762024, ['Person: 84%']]]
[[427, 856, 1224, 1392, 94.1999077796936, ['Person: 94%']], [391, 785, 693, 1094, 82.52095580101013, ['Dumpster: 82%']]]
[[418, 864, 1213, 1388

In [17]:
print(bounding_box)

[[[431, 818, 610, 1104, 99.73264336585999, 'Person: 92%'], [461, 930, 1380, 1611, 92.84646511077881, 'Person: 92%']], [[428, 819, 608, 1107, 99.59614276885986, 'Person: 95%'], [448, 908, 1376, 1620, 95.00068426132202, 'Person: 95%']], [[426, 819, 611, 1105, 99.36834573745728, 'Person: 96%'], [454, 934, 1379, 1607, 96.83652520179749, 'Person: 96%']], [[418, 819, 619, 1099, 98.99320602416992, 'Person: 95%'], [447, 917, 1373, 1590, 95.01233100891113, 'Person: 95%']], [[421, 821, 618, 1102, 98.95820021629333, 'Person: 98%'], [440, 922, 1374, 1588, 98.31085205078125, 'Person: 98%']], [[417, 814, 616, 1099, 99.41073656082153, 'Person: 98%'], [444, 926, 1369, 1588, 98.35406541824341, 'Person: 98%']], [[419, 814, 619, 1100, 99.40218925476074, 'Person: 99%'], [444, 919, 1364, 1584, 99.10041093826294, 'Person: 99%']], [[430, 819, 616, 1103, 99.62965250015259, 'Person: 99%'], [454, 916, 1361, 1580, 99.12697076797485, 'Person: 99%']], [[434, 821, 615, 1098, 99.73179697990417, 'Person: 99%'], [458,